<a href="https://colab.research.google.com/github/AjitNikam1/Red-Wine-quality/blob/master/RedWineQualityPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Machine Learning Project -Red Wine quality
In this problem, the goal is to model the quality of red wine based on its physical properties. This dataset consists of attributes related to the red wine variant of the Portuguese "Vinho Verde" wine

###Import Required Packages

In [0]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Read the Data from .csv file 

In [21]:
# url = 'https://github.com/AjitNikam1/Red-Wine-quality/blob/master/winequality-red.csv'
RedWineData = pd.read_csv('https://github.com/AjitNikam1/Red-Wine-quality/blob/master/winequality-red.csv',error_bad_lines=False)
RedWineData.head()
RedWineData.info()
RedWineData.shape
RedWineData.columns

b'Skipping line 75: expected 1 fields, saw 7\nSkipping line 124: expected 1 fields, saw 3\nSkipping line 141: expected 1 fields, saw 9\nSkipping line 142: expected 1 fields, saw 3\nSkipping line 166: expected 1 fields, saw 4\nSkipping line 170: expected 1 fields, saw 2\nSkipping line 172: expected 1 fields, saw 2\nSkipping line 173: expected 1 fields, saw 2\nSkipping line 174: expected 1 fields, saw 2\nSkipping line 175: expected 1 fields, saw 2\nSkipping line 176: expected 1 fields, saw 2\nSkipping line 177: expected 1 fields, saw 2\nSkipping line 178: expected 1 fields, saw 2\nSkipping line 179: expected 1 fields, saw 2\nSkipping line 180: expected 1 fields, saw 2\nSkipping line 184: expected 1 fields, saw 2\nSkipping line 185: expected 1 fields, saw 2\nSkipping line 191: expected 1 fields, saw 2\nSkipping line 199: expected 1 fields, saw 4\nSkipping line 205: expected 1 fields, saw 2\nSkipping line 210: expected 1 fields, saw 2\nSkipping line 211: expected 1 fields, saw 2\nSkipping 

,<!DOCTYPE html>
0,"<html lang=""en"">"
1,<head>
2,"<meta charset=""utf-8"">"
3,"<link rel=""dns-prefetch"" href=""https://githu..."
4,"<link rel=""dns-prefetch"" href=""https://avata..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6960 entries, 0 to 6959
Data columns (total 1 columns):
<!DOCTYPE html>    6960 non-null object
dtypes: object(1)
memory usage: 54.5+ KB


(6960, 1)

Index(['<!DOCTYPE html>'], dtype='object')